In [ ]:
import com.google.gson.GsonBuilder
import hu.akos.hollo.szabo.math.Permutation
import hu.raven.puppet.logic.statistics.edgeHistogramMatrix
import hu.raven.puppet.logic.statistics.mapStates
import hu.raven.puppet.utility.PermutationTypeAdapter
import hu.raven.puppet.model.state.BacteriophageAlgorithmState
import hu.raven.puppet.model.utility.math.GraphEdge
import java.io.File
import java.time.LocalDate
import java.time.LocalDateTime
import hu.raven.puppet.utility.LocalDateTimeTypeAdapter
import hu.raven.puppet.utility.LocalDateTypeAdapter
import com.google.gson.reflect.TypeToken
import hu.raven.puppet.job.experiments.Configuration

In [ ]:
val gson = GsonBuilder()
    .registerTypeAdapter(Permutation::class.java, PermutationTypeAdapter)
    .registerTypeAdapter(LocalDateTime::class.java, LocalDateTimeTypeAdapter)
    .registerTypeAdapter(LocalDate::class.java, LocalDateTypeAdapter)
    .create()

In [ ]:
val costTimeSeriesFile = File("D:\\Research\\Results\\extractions\\costTimeSeriesWithBiggerPopulation.json")

In [ ]:
data class ConfigurationWithTimeSeries(
    val configuration: Configuration,
    val timeAxes: LongArray,
    val indexAxes: IntArray,
    val valuesAxes: IntArray,
)

var records = costTimeSeriesFile.useLines { lines ->
    val typeToken = object : TypeToken<Pair<Configuration, List<Pair<Long, Int>>>>() {}.type
    lines
        .map {
            gson.fromJson<Pair<Configuration, List<Pair<Long, Int>>>>(it, typeToken)
        }
        .map { configAndTimeSeries ->
            Pair(
                configAndTimeSeries.first,
                configAndTimeSeries.second
                    .withIndex()
                    .filter {
                        it.index == 0 || configAndTimeSeries.second[it.index - 1].second > it.value.second
                    }
            )
        }
        .map {
            ConfigurationWithTimeSeries(
                configuration = it.first,
                timeAxes = it.second.map { it.value.first }.toLongArray(),
                indexAxes = it.second.map { it.index }.toIntArray(),
                valuesAxes = it.second.map { it.value.second }.toIntArray(),
            )
        }
        .toList()
}

In [ ]:
%use kandy

In [ ]:
val indexLimit = records.minOf {
        it.indexAxes.asList().max()
}
val timeLimit = records.minOf {
        it.timeAxes.asList().max()
}

In [ ]:
fun <T> valueSequencer(value: T, count: Int) =
    sequence {
        repeat(count) { yield(value) }
    }

plot {
    records
        .asSequence()
        .filter { it.configuration.sizeOfPopulation == 16 }
        .filter { it.configuration.fileName == "instance6.json" }
        .forEachIndexed { index, record ->
            line {
                val indexCount = record.indexAxes.count { it <= indexLimit }
                color(
                    valueSequencer(
                        record.configuration.boostLuckyCount,
                        indexCount
                    ).asIterable()
                ) {
                    scale = continuous(Color.BLUE..Color.RED)
                }
                x(record.indexAxes.take(indexCount))
                y(record.valuesAxes.take(indexCount))
            }
        }
}